In [1]:
%jsroot on
TFile *File = new TFile("calibration.root"); 
TCanvas* c = new TCanvas("c", "c", 700, 700); 
TTree *Adjust = (TTree*) File -> Get("Adjust"); 
TTree *FitData = (TTree*) File -> Get("FitData"); 

# 调整道址的$\chi^2/ndf$展示。

In [2]:
c -> Clear(); 
TGraph *kbFitChiNdf = (TGraph*) File -> Get("kbFitChiNdf"); 
kbFitChiNdf -> Draw("ALP"); 
c -> Draw(); 

# 调整道址后的总靶

In [3]:
c -> Clear(); 
TH1D *AllAdjusted = (TH1D*) File -> Get("AllAdjusted"); 
AllAdjusted -> Draw(); 
c -> Draw(); 

# 半高宽

In [4]:
c -> Clear(); 
TGraph *FWHM[4]; 
TMultiGraph *mg = new TMultiGraph();
for(Int_t i = 0; i < 4; i++){
    FWHM[i] = (TGraph*) File -> Get(Form("FWHM%i",i)); 
    FWHM[i] -> SetLineColor(i + 1); 
    mg -> Add(FWHM[i]); 
}
mg -> Draw("AL*"); 
c -> Draw(); 

# 我本来想画一个TH2D的对比图，但是很明显，这不行而我还不知道为啥，或许是Draw的问题，我改为生成两个图像。

In [14]:
TCanvas *c1 = new TCanvas("c1", "c1", 1200, 500); 
c1 -> Divide(2, 1); 

c1 -> cd(1); 
Adjust -> Draw("pe:PID>>(48,0,48,800,0,1600)","","colz"); 

c1 -> cd(2); 
FitData -> Draw("Fitpe:PID>>(48,0,48,800,0,11.2)","","colz"); 

c1 -> Draw(); 

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1


# 而分开画就没有问题了

In [6]:
c -> Clear(); 
TTree *Adjust = (TTree*) File -> Get("Adjust"); 
Adjust -> Draw("pe[0]:PID>>(48,0,48,800,0,1600)","","colz"); 
c -> Draw(); 

In [13]:
c -> Clear(); 
c -> SetLogz(0); 
TTree *FitData = (TTree*) File -> Get("FitData"); 
FitData -> Draw("Fitpe[0]:PID>>(48,0,48,800,0,11.2)","","colz"); 
c -> Draw(); 

# 决定再尝试一下，先写入两个TH2D中的情况。

In [2]:
Int_t PID[48], Ope; 
Double_t Fitpe[48], pe[48]; 
FitData -> SetBranchAddress("PID", PID); 
FitData -> SetBranchAddress("Ope", &Ope); 
FitData -> SetBranchAddress("Fitpe", Fitpe); 
Adjust -> SetBranchAddress("pe", pe); 

TH2D *hFitData = new TH2D("hFitData", "hFitData", 48, 0, 48, 800, 0, 11.2); 
TH2D *hAdjust  = new TH2D("hAdjust" , "hAdjust" , 48, 0, 48, 800, 0, 1600); 

for(Long64_t j = 0; j < FitData -> GetEntries(); j++){
    FitData -> GetEntry(j); 
    Adjust -> GetEntry(j); 
    for(Int_t i = 0; i < Ope; i++){
        hFitData -> Fill(PID[i], Fitpe[i]); 
        hAdjust -> Fill(PID[i], pe[i]); 
    }
}
TCanvas *c1 = new TCanvas("c1", "c1", 1500, 700); 
c1 -> Divide(2, 1); 
c1 -> cd(1); 
hAdjust -> Draw("colz"); 
c1 -> cd(2); 
hFitData -> Draw("colz"); 
c1 -> Draw(); 